In [2]:
!pip install langchain langchain_community openai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
# Run once to materialize the prompt files ------------
detector_text = """<<< # Domain Identification Prompt (ReAct + Reflexion style)

---
## System message (immutable)
```
You are a *domain‑identification agent*.
Input: a JSON array of objects, each object having exactly two keys:
  • "column": string – the column name
  • "dtype"  : string – a coarse data type such as "numeric", "categorical", "date", "text".

Goal: return a ranked list of candidate knowledge domains *plus* the domain‑specific concepts / jargon that are *not already present* verbatim in the column list.

Use a ReAct loop:
 1. **Think** – reason about what real‑world domains typically contain variables like these dtypes / names.
 2. **Act**   – (optional) call tools such as `search_examples(domain)` to recall external hints.
 3. **Reflect** – after a tentative answer, invoke the *Evaluator* to critique whether any proposed concept is merely an existing column name. If so, revise.

Return JSON with this exact schema:
{
  "domains": [
    {
      "name"        : string,          # e.g. "Finance"
      "confidence"  : number 0‑1,
      "concepts"    : [string,...]    # at least 3 unique jargon terms NOT identical (case‑insensitive) to any column name
    },
    ...
  ]
}

Always produce **exactly 5** domain objects.  Order by descending confidence.
```

---
## Few‑shot exemplars (abbreviated)

> ### Example 1
> **Input**  
> ```json
> [{"column": "loan_amount", "dtype": "numeric"}, {"column": "interest_rate", "dtype": "numeric"}, {"column": "term_months", "dtype": "numeric"}]
> ```
> **Thought**: Numeric loan variables → Finance or Banking.
> **Reflection**: Concepts must not repeat column names.
> **Answer**  
> ```json
> {"domains":[{"name":"Banking","confidence":0.9,"concepts":["amortization","principal","APR"]}, …]}
> ```

### Example 2
**Input**
```json
[
  {"column": "age", "dtype": "numeric"},
  {"column": "height_cm", "dtype": "numeric"},
  {"column": "weight_kg", "dtype": "numeric"},
  {"column": "blood_pressure", "dtype": "numeric"},
  {"column": "cholesterol_level", "dtype": "numeric"}
]
```
**Think**: Vital‑sign style metrics suggest Healthcare / Clinical / Physiology domains.

**Reflect** *(Evaluator)*: Ensure jargon isn’t merely “age”, “height”, etc. Swap in higher‑level concepts.

**Answer**
```json
{
  "domains": [
    {
      "name": "Healthcare",
      "confidence": 0.88,
      "concepts": ["BMI", "systolic", "diastolic"]
    },
    {
      "name": "Exercise Science",
      "confidence": 0.6,
      "concepts": ["VO2max", "basal_metabolic_rate", "cardiorespiratory_fitness"]
    },
    {
      "name": "Nutrition",
      "confidence": 0.55,
      "concepts": ["macronutrient", "lipid_profile", "HDL"]
    },
    {
      "name": "Public Health",
      "confidence": 0.42,
      "concepts": ["epidemiology", "risk_factor", "population_health"]
    },
    {
      "name": "Gerontology",
      "confidence": 0.25,
      "concepts": ["frailty_index", "chronological_age", "longevity"]
    }
  ]
}
```

 >>>"""
retrieval_text = """<<< # Formula‑Retrieval Prompt (ReAct‑style)

---

## System message (immutable)

```
You are a *formula‑retrieval agent*.
Input is a JSON object with two keys:
  • "domain"      : string  – the high‑confidence domain selected in Step 1.
  • "concepts"    : [string] – 3‑5 domain‑specific jargon terms NOT identical to dataset column names.

Objective:
 1. Search authoritative public sources (Wikipedia) for mathematical or accounting formulas that *connect* these concepts.
 2. Return up to **5** formulas most relevant to the input domain.
 3. For each formula produce:
      • LaTeX string (formula_tex)
      • Plain‑English explanation (formula_text)
      • Required symbolic variables as an array (required_vars) – keep symbols short, e.g. [R,Cost,Profit]
      • source_pageid (integer) – Wikipedia pageid you extracted from.

Constraints:
 • Skip trivial identities (e.g. Revenue = Revenue).
 • Do not invent original formulas; cite existing published ones.
 • If no formula found after reasonable effort, return exactly: `{ "no_formula": true }` and stop.

Tools available (ReAct pattern):
 1. `wikipedia_search`: "search <query>"  → returns JSON list of {pageid,title,snippet}
 2. `wikipedia_extract`: "extract <pageid> <heading_keyword>" → returns raw wikitext of that section.

Think / Act / Reflect cycle:
 • **Think** what concept pair might yield a known formula.
 • **Act** call search/extract tools.
 • **Reflect** decide if formula is relevant and unique; store it.
 • Loop until you have ≤5 good formulas or hit search depth limit (10 calls).

Output JSON schema:
{
  "formulas": [
    {
      "formula_tex"   : string,
      "formula_text"  : string,
      "required_vars" : [string,...],
      "source_pageid" : integer
    },
    ...  (max 5)
  ]
}
```

---

## Few‑shot exemplar

### Example

**Input**

```json
{
  "domain": "Finance",
  "concepts": ["revenue", "profit", "transaction", "customer"]
}
```

**(abbreviated transcript)**

```
Thought: Formulas in managerial accounting often link revenue, profit and cost. Start with search.
Act: search revenue profit equation
... (tool returns pageid 12345 "Profit (accounting)")
Act: extract 12345 formula
Reflect: Found "Profit = Revenue - Costs". keep.
Thought: another useful ratio is profit margin.
Act: search profit margin formula
... (returns pageid 67890 ...)
...
```

**Answer**

```json
{
  "formulas": [
    {
      "formula_tex": "\\text{Profit}=\\text{Revenue}-\\text{Costs}",
      "formula_text": "Profit equals total revenue minus total costs.",
      "required_vars": ["Profit","Revenue","Costs"],
      "source_pageid": 12345
    },
    {
      "formula_tex": "\\text{Profit Margin}=\\frac{\\text{Profit}}{\\text{Revenue}}",
      "formula_text": "The profit margin is the ratio of profit to revenue.",
      "required_vars": ["Profit","Revenue"],
      "source_pageid": 67890
    }
  ]
}
```

 >>>"""

with open("domain_detector_prompt.txt", "w", encoding="utf-8") as f:
    f.write(detector_text)

with open("formula_retrieval_prompt.txt", "w", encoding="utf-8") as f:
    f.write(retrieval_text)

In [4]:
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
import requests, json, shelve

############################################################################
# 1. Domain detector
############################################################################
detector_prompt = PromptTemplate.from_file("domain_detector_prompt.txt")
detector_chain  = LLMChain(llm=OpenAI(model="gpt-4o-mini"), prompt=detector_prompt)

schema = [
    {"column": "Quarter",            "dtype": "text"},
    {"column": "number_Customers",   "dtype": "text"},
    {"column": "Total_Transactions", "dtype": "text"},
    {"column": "Revenue",            "dtype": "text"},
    {"column": "Profit",             "dtype": "text"},
]
detector_out = detector_chain.run(columns=json.dumps(schema, indent=2))
domain_info  = json.loads(detector_out)          # {"domains":[...],"concepts":[...]}

############################################################################
# 2. Wiki tools with simple file-cache
############################################################################
CACHE = shelve.open("wiki_cache.db")

class WikiSearchTool(BaseTool):
    name, description = "wikipedia_search", "search <query> on en.wikipedia"
    def _run(self, query):
        url = "https://en.wikipedia.org/w/api.php"
        params = {"action":"query","list":"search","srsearch":query,"format":"json"}
        key   = f"search::{query}"
        if key not in CACHE:
            CACHE[key] = requests.get(url, params=params).json()
        return CACHE[key]["query"]["search"][:5]

class WikiExtractTool(BaseTool):
    name, description = "wikipedia_extract", "extract <pageid> <heading_keyword> text"
    def _run(self, pageid_heading):
        pageid, heading = pageid_heading.split(maxsplit=1)
        key   = f"extract::{pageid}::{heading}"
        if key not in CACHE:
            url = "https://en.wikipedia.org/w/api.php"
            params = {"action":"parse","pageid":pageid,"prop":"wikitext","format":"json"}
            CACHE[key] = requests.get(url, params=params).json()
        # very naïve heading filter:
        return CACHE[key]["parse"]["wikitext"]["*"]

############################################################################
# 3. Initialize the ReAct agent
############################################################################
tools   = [WikiSearchTool(), WikiExtractTool()]
react_prompt = PromptTemplate.from_file("formula_retrieval_prompt.txt")
agent   = initialize_agent(
            tools=tools,
            llm=OpenAI(model="gpt-4o-mini"),
            agent=AgentType.REACT_DESCRIPTION,
            prompt=react_prompt,
            verbose=True,
          )

retrieval_input = {
    "domain":   domain_info["domains"][0]["name"],
    "concepts": domain_info["concepts"]
}
formulas_json = agent.run(json.dumps(retrieval_input, indent=2))
print(formulas_json)
############################################################################


/Users/zhangran/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/p4/f1848b4x3815qpp5kmf2j66w0000gn/T/ipykernel_19674/2242617425.py:10: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  detector_chain  = LLMChain(llm=OpenAI(model="gpt-4o-mini"), prompt=detector_prompt)


ValidationError: 1 validation error for OpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {}, 'nam...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error